In [1]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [2]:
import numpy as np

In [15]:
x= [[1,1,1],[1,1,1]]
y=[[1,1,1],[1,1,1]]

x_tensor=torch.tensor(x)
y_tensor=torch.tensor(y)

print(x_tensor.shape,'\n', y_tensor.shape)

torch.Size([2, 3]) 
 torch.Size([2, 3])


In [19]:
#Matrix Multiplication 2x3 . 2x3
print(torch.matmul(x_tensor,y_tensor.T))

#Elementwise Multiplication 2x3
torch.mul(x_tensor,y_tensor)
torch.mul(y_tensor,x_tensor)

tensor([[3, 3],
        [3, 3]])


tensor([[1, 1, 1],
        [1, 1, 1]])

In [80]:
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [81]:
train_dataloader = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_dataloader = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [174]:
train_data= DataLoader(train_dataloader, batch_size=64)
test_data= DataLoader(test_dataloader, batch_size=64)

In [175]:
train_data.to(device)

AttributeError: 'DataLoader' object has no attribute 'to'

In [242]:
class NeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            # nn.Linear(512,512),
            # nn.ReLU(),
            nn.Linear(512,10)
            )

    def forward(self, x):
        x=self.flatten(x)
        logits= self.linear_relu_stack(x)
        return logits



In [243]:
def train_loop(dataloader, model, loss_fn, optimizer):

    #sets the model in training mode, behaves differently during eval
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    #938 batches
    for batch, (X,y) in enumerate(dataloader):
        pred=model(X)
        loss=loss_fn(pred, y)
        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        #backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f'loss:{loss:>7f} [{current:>5d}/{size:>5d}]')
    
    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")


def test_loop(dataloader, model, loss_fn):
 
    #evaluation mode
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")




In [244]:
model=NeuralNetwork()


#Hyperparameters and loss fn
learning_rate=1e-3
batch_size=64

loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 25
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_data, model, loss_fn, optimizer)
    test_loop(test_data, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.293062 [    0/60000]
loss:2.228018 [ 6400/60000]
loss:2.149761 [12800/60000]
loss:2.126793 [19200/60000]
loss:2.038784 [25600/60000]
loss:1.960079 [32000/60000]
loss:1.969828 [38400/60000]
loss:1.871575 [44800/60000]
loss:1.857759 [51200/60000]
loss:1.759863 [57600/60000]
Train Error: 
 Accuracy: 48.2%, Avg loss: 2.014957 

Test Error: 
 Accuracy: 61.1%, Avg loss: 1.759875 

Epoch 2
-------------------------------
loss:1.807440 [    0/60000]
loss:1.751526 [ 6400/60000]
loss:1.610001 [12800/60000]
loss:1.656354 [19200/60000]
loss:1.510711 [25600/60000]
loss:1.482076 [32000/60000]
loss:1.493091 [38400/60000]
loss:1.407451 [44800/60000]
loss:1.424914 [51200/60000]
loss:1.318050 [57600/60000]
Train Error: 
 Accuracy: 64.5%, Avg loss: 1.529797 

Test Error: 
 Accuracy: 64.5%, Avg loss: 1.344376 

Epoch 3
-------------------------------
loss:1.418686 [    0/60000]
loss:1.386126 [ 6400/60000]
loss:1.218452 [12800/60000]
loss:1.317475 [19200/60000